In [36]:
import requests as rq
import pandas as pd
import time

In [37]:
"""
Với cookies thì ta có thể cần hoặc không 
- Cần trong trường hợp cần truy cập vào lịch sử mua hàng, giỏ hàng,... -> dữ liệu riêng tư
- Không cần trong trường hợp dữ liệu công khai như: list các tựa sách,.. 

"""

cookies = { 
    '_trackity': '55401861-5f62-8669-0c44-7cdc73fb7301', 
    'TOKENS': '{%22access_token%22:%22B4ld6IYTZu5jrnQUwOCxP9a0MXmJoy8E%22}', 
    '_gcl_gs': '2.1.k1$i1759298855$u206005270',
    '_ga': 'GA1.1.253010683.1759122115',
    'delivery_zone': 'Vk4wMzQwMjQwMTM=', 
    '_gcl_aw': 'GCL.1759298861.CjwKCAjw_-3GBhAYEiwAjh9fUC7GSyS5yeT2fQH0Hto_gsT4TXrwxCFZy2gVq8T-uGhbNGp4_zs43BoCIPcQAvD_BwE', 
    '_gcl_au': '1.1.308886142.1759122119',
    '_hjSession_522327': 'eyJpZCI6IjgzOWNhNWUzLTgwZjMtNDY4OS1hMGYxLTc4N2JhNDkxZjdkYSIsImMiOjE3NTkyOTg4NjM4NDgsInMiOjAsInIiOjAsInNiIjowLCJzciI6MCwic2UiOjAsImZzIjowLCJzcCI6MH0=', 
    '_tt_enable_cookie': '1',
    '_ttp': '01K69XJPTGEKE7CSE559TCFT3S_.tt.1', 
    '__utm': 'source%3Dgoogle%7Cmedium%3Dcpc%7Ccampaign%3DSEA_NBR_GGL_PMA_DAP_ALL_VN_ALL_UNK_UNK_C.PMAX_X.22956852689_Y.185206935272_V._W.DT_A._O.CIR', 
    '__iid': '749', 
    '__su': '0', 
    '_fbp': 'fb.1.1759122119865.100853033443494250', 
    '__RC': '4', '__R': '1', 'tiki_client_id': '253010683.1759122115', 
    '_hjSessionUser_522327': 'eyJpZCI6IjA1Zjc0NjEyLWFlNjktNTJmOC05OWVhLWNhYWFjMzlhMDczMSIsImNyZWF0ZWQiOjE3NTkxMjIxMTkzMTQsImV4aXN0aW5nIjp0cnVlfQ==',
    '_ga_S9GLR1RQFJ': 'GS2.1.s1759298857$o14$g1$t1759298993$j60$l0$h0',
    'amp_99d374': 'Wyj8tBkgXYCNqfmSqyPtxk...1j6f649jb.1j6f68fro.gt.kc.159',
    '__uif': '__uid%3A1136825641523629859%7C__ui%3A1%252C4%7C__create%3A1743682564', 
    '__tb': '0', '__IP': '1891973936', 'ttcsid': '1759298864068::gVp9JOjJ7ayehUZhW0AR.13.1759298869709.0',
    'ttcsid_D031T03C77UFNM54TPR0': '1759298864067::zW5g8DQZcC6QiCVZdpl5.13.1759298869710.0'
}

headers = {
    # Cần User-Agent vì một số server chặn bot nếu như UA không hợp lệ 
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
    'Accept' : 'application/json, text/plain, */*', # Cho server biết data type mà mình muốn trả về  
    'Accept-Language' : 'vi-VN,vi;q=0.9,fr-FR;q=0.8,fr;q=0.7,en-US;q=0.6,en;q=0.5', # Yêu cầu ngôn ngữ data được trả về
    'Referer' : 'https://tiki.vn/nha-sach-tiki/c8322?from=header_keyword', # Tránh việc server nghĩ là bot, khi đó sẽ hiện lỗi 403 (forbidden) 
    'X-Guest-Token' : 'B4ld6IYTZu5jrnQUwOCxP9a0MXmJoy8E', # Giúp server nhận diện được khách không đăng nhập mà chỉ truy cập xem hàng hóa
}

params = { 
    'limit' : '40', # Số sản phẩm tối đa mỗi trang 
    #'include' : 'advertisement', bỏ qua để khi trả dữ liệu chỉ chứa sản phẩm chính 
    'category' : '8322', # Mã của danh mục 
    'page' : '1', # Trang cần lấy dữ liệu 
    'urlKey' : 'nha-sach-tiki' # Giúp nhận diện có khớp với mã của danh mục hay không
}


In [38]:
# Sử dụng GET trong requests để lấy dữ liệu từ trang web 
response = rq.get('https://tiki.vn/api/personalish/v1/blocks/listings', headers= headers, params= params)
"""
    Tại sao lại cần truyền params, headers? 
    Headers : Một số trang web thường chặn nếu như cảm thấy không giống trình duyệt thật 
    Params : Nếu không truyền params thì sẽ chỉ lấy được giá trị mặc định của server ('page' : '1')
    vậy nên cần truyền Params để crawl được nhiều data hơn 

"""
# Kiểm tra mã trạng thái (status code) 
if response.status_code == 200: 
    print("Truy cập thành công")
else: 
    print("Truy cập không thành công")

Truy cập thành công


In [ ]:
books_information = [] # Tạo một list rỗng để chứa các ID sách
for i in range(1, 16): # Cho số trang i chạy từ 1 - 10
    params['page'] = i
    # Gửi requests để lấy dữ liệu của các trang tiếp theo
    response = rq.get('https://tiki.vn/api/personalish/v1/blocks/listings', headers=headers, params=params)
    for books in response.json().get('data', []): # Trả về là [] nếu không có dữ liệu
        author = "None" # Nếu không có tên tác giả thì mặc định là None
        for badge in books.get('badges_new', []):
            if badge.get('code') == 'brand_name':
                author = badge.get('text')

        # Không dùng lặp for vì "visible_impression_info" là 1 dict
        # Nếu không có giá trị trả về {}
        type_of_book = books.get('visible_impression_info', {})
        types = "None"
        # Nếu type_of_book và key bên trong tên amplitude không None 
        if type_of_book is not None and type_of_book.get('amplitude') is not None:
            types = type_of_book.get('amplitude').get('category_l3_name') # Lấy key tên category_l3_name chứa dữ liệu cần

        # Kiểm tra số lượt bán
        quality_sold = books.get('quantity_sold')
        sold = "None" # Mặc định là None
        if quality_sold is not None:
            sold = quality_sold.get('value')

        # Đường dẫn lấy chi tiết của các sản phẩm theo từng ID
        book_detail_path = f"https://tiki.vn/api/v2/products/{books.get('id')}?platform=web&spid={books.get('id')}&version=3"
        # Gửi requests để lấy dữ liệu chi tiết từ đường dẫn
        book_detail_response = rq.get(book_detail_path, cookies=cookies, headers=headers, timeout=10, allow_redirects=False)
        try:
            book_detail_json = book_detail_response.json()
        except ValueError:
            # Nếu không phải JSON, bỏ qua sản phẩm này
            continue

        # Gán giá trị mặc định là "None"
        version = publisher = "None"
        dimensions = dich_gia = "None"
        cover = num_of_page = "None"
        manu = "None"
        # Lặp qua các phần tử trong list, lấy giá trị của key tên specifications
        for spec in book_detail_json.get('specifications', []):
            for attribute in spec.get('attributes', []):
                code = attribute.get('code')
                if code == 'book_version':
                    version = attribute.get('value')
                if code == 'publisher_vn':
                    publisher = attribute.get('value')
                if code == 'dimensions':
                    dimensions = attribute.get('value')
                if code == 'dich_gia':
                    dich_gia = attribute.get('value')
                if code == 'book_cover':
                    cover = attribute.get('value')
                if code == 'number_of_page':
                    num_of_page = attribute.get('value')
                if code == 'manufacturer':
                    manu = attribute.get('value')
                    
        # Thêm các thông tin của sách vào book_information
        books_information.append({
            'ID': books.get('id', "None"),
            'Tên sách': books.get('name', "None"),
            'Tên tác giả': author,
            'Thể loại sách': types,
            'Số trang': num_of_page,
            'Nhà xuất bản': manu,
            'Công ty phát hành': publisher,
            'Dịch giả': dich_gia,
            'Loại bìa': cover,
            'Loại sách': version,
            'Kích thước': dimensions,
            'Giá gốc (VNĐ)': books.get('original_price', "None"),
            'Giá hiện tại (VNĐ)': books.get('price', "None"),
            'Khuyến mãi': books.get('discount_rate', "None"),
            'Đã bán': sold,
            'Đánh giá': books.get('rating_average', "None"),
            'Lượt đánh giá': books.get('review_count', "None")
        })

# Tạo DataFrame và lưu ra file Excel
if books_information:
    df = pd.DataFrame(books_information)
    df['Số trang'] = pd.to_numeric(df['Số trang'], errors='coerce') # Chuyển các số trang đang ở dạng văn bản thành số nguyên
    df.to_excel('Books_Informations.xlsx', index=False)
else:
    print("Không có dữ liệu sách để lưu.")